In [1]:
import os  
import sys  
import threading  
import time  
import queue  

In [2]:
# 定义全局变量  
names_file = "names.txt"  
chapter_folder = "shz"  
line_queue = queue.Queue(maxsize=10)  # 假设队列的最大尺寸是10  
condition = threading.Condition()  
progress = 0  

In [3]:
# 从names.txt中读取主要人物列表  
def get_main_characters():  
    with open(names_file, 'r', encoding='utf-8') as f:  
        return [line.strip() for line in f]  
  
main_characters = get_main_characters()  

In [4]:
# 第一个子线程：查找主要人物并放入line_queue  
def thread_1_task():  
    global progress  
    with condition:  
        for chapter_num in range(1,122):
            chapter_file = f'水浒传第{chapter_num}回.txt'
            if chapter_file.endswith(".txt"):  
                with open(os.path.join(chapter_folder, chapter_file), 'r', encoding='utf-8') as f:  
                    for line_num, line in enumerate(f, start=1):  
                        for character in main_characters:  
                            if character in line:  
                                line_data = [chapter_num, line_num, line]  
                                line_queue.put(line_data)  
                                with condition:  
                                    progress += 1  
                                    condition.notify_all()  
                                time.sleep(0.01)  # 模拟处理延迟  
                                if line_queue.full():  
                                    while line_queue.full():  
                                        condition.wait() 

In [5]:
# 第二个子线程：从line_queue读取数据并写入文件  
def thread_2_task():  
    global progress  
    while True:  
        with condition:  
            while line_queue.empty():  
                condition.wait()  
            while not line_queue.empty():  
                chapter_num, line_num, line = line_queue.get()  
                for character in main_characters:  
                    if character in line:  
                        with open(f"{character}.txt", 'a', encoding='utf-8') as f:  
                            f.write(f"{chapter_num}|{line_num}|{line}\n")  
                line_queue.task_done()  
                progress += 1  
                condition.notify_all()  
            if progress >= 121 * 1000:
                break  
            time.sleep(0.01)  # 模拟处理延迟  

In [6]:
# 第三个子线程：进度监控  
def thread_3_task():  
    global progress  
    chapter_num, line_num, line = line_queue.get() 
    while chapter_num < 121: 
        chapter_num, line_num, line = line_queue.get() 
        sys.stdout.write("\r正在处理水浒传第{}回Progress: {}%".format(chapter_num,(chapter_num/ 121)*100) ) # 输出进度百分比  
        sys.stdout.flush()  
        time.sleep(1)  # 每秒更新一次进度显示  
    print("\nTask Completed!")  

In [ ]:
# 创建并启动线程
thread_1 = threading.Thread(target=thread_1_task)  
thread_2 = threading.Thread(target=thread_2_task)  
thread_3 = threading.Thread(target=thread_3_task)  
thread_1.start()  
thread_2.start()  
thread_3.start()

正在处理水浒传第3回Progress: 2.479338842975207%

正在处理水浒传第121回Progress: 100.0%55371900827%
Task Completed!
